In [ ]:
!pip install tensorflow-text

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aminesedkaoui/epinions-ratings-500k
! kaggle datasets download -d aminesedkaoui/epinions-user-item-matrix-75k

In [5]:
from scipy.sparse import csr_matrix
import tensorflow as tf 
import numpy as np
import tensorflow_hub as hub
import pandas as pd
import random
from keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [6]:
!unzip "/content/epinions-ratings-500k.zip" -d "/content/"
!unzip "/content/epinions-user-item-matrix-75k.zip" -d "/content/"

Archive:  /content/epinions-ratings-500k.zip
  inflating: /content/epinions_rating_500k.csv  
Archive:  /content/epinions-user-item-matrix-75k.zip
  inflating: /content/epinions_user_item_filled_75K_matrix_100k_leaning.csv  


In [7]:
df = pd.read_csv('/content/epinions_rating_500k.csv')
df = df.drop(['Unnamed: 0'],axis=1)
df.head()

,Item_ID,User_ID,rating
0,139431556,1312460676,5
1,139431556,204358,5
2,139431556,368725,5
3,139431556,277629,5
4,139431556,246386,5


In [8]:
def DatasetToUserItemDataFrame(dataframe,userID,itemID,rating):
  #Setting new item IDs from string to int 
  itemKeys = [] 
  i = 0
  for item in dataframe[itemID].value_counts(sort=False):
    temp = np.full((item),i)
    itemKeys = np.append(itemKeys,temp)
    i += 1

  #Setting new user IDs from string to int
  userKeysDic = {}
  userKeys = np.zeros((dataframe[userID].size))
  i = 0
  for user in dataframe[userID].unique():
    userKeysDic[user] = i
    i += 1
  i = 0
  for user in dataframe[userID]:
    userKeys[i] = userKeysDic[user]
    i += 1

  #Converting arrays from float to int 
  userKeys = userKeys.astype(int)
  itemKeys = itemKeys.astype(int)

  
  user_item = csr_matrix((dataframe[rating].values.astype(int),(userKeys,itemKeys))) #Creating sparse matrix
  user_item_matrix = user_item.toarray() #Converting sparse matrix into array
  df_user_item = pd.DataFrame(user_item_matrix,index = dataframe[userID].unique()  ,columns = dataframe[itemID].unique() ) 

  return user_item,df_user_item

In [9]:
def preprocess_df(df):
  users = df['User_ID'].unique()
  items = df['Item_ID'].unique()
  df_train = df.copy()

  New_User_IDs = random.sample(range(10000,50000),df['User_ID'].nunique())
  New_Item_IDs = random.sample(range(100000,200000),df['Item_ID'].nunique())
  i = 0
  for d in users :
    df_train['User_ID'].replace({d : New_User_IDs[i]}, inplace=True)
    i+=1

  i = 0
  for d in items :
    df_train['Item_ID'].replace({d : New_Item_IDs[i]}, inplace=True)
    i+=1
  
  df_train['one']=df_train['rating'].apply(lambda x: 1 if x==1 else 0)
  df_train['two']=df_train['rating'].apply(lambda x: 1 if x==2 else 0)
  df_train['three']=df_train['rating'].apply(lambda x: 1 if x==3 else 0)
  df_train['four']=df_train['rating'].apply(lambda x: 1 if x==4 else 0)
  df_train['five']=df_train['rating'].apply(lambda x: 1 if x==5 else 0)
  df_train['six']=df_train['rating'].apply(lambda x: 1 if x==6 else 0)

  return df_train

In [10]:
from numpy.core.fromnumeric import size
#GMF

def Create_GMF(num_users,num_items,SIZE_):

  input_userID = layers.Input(shape=[1], name='user_ID')
  input_itemID = layers.Input(shape=[1], name='item_ID')

  user_emb_GMF = layers.Embedding(num_users, SIZE_, name='user_emb_GMF')(input_userID)
  item_emb_GMF = layers.Embedding(num_items, SIZE_, name='item_emb_GMF')(input_itemID)

  u_GMF = layers.Flatten()(user_emb_GMF)
  i_GMF = layers.Flatten()(item_emb_GMF)

  dot_layer = layers.Multiply()([u_GMF, i_GMF])

  out_layer = layers.Dense(6, activation='softmax', name='output')(dot_layer)

  GMF = tf.keras.Model([input_userID, input_itemID], out_layer)
  
  return GMF

def Train_GMF(model,X_train,Y_train,nbrEpochs):
  model.compile(optimizer = 'adam',
                    loss = tf.keras.losses.categorical_crossentropy ,
                    metrics=['accuracy'])
  model.fit(X_train,Y_train,epochs = nbrEpochs)
  return model

def user_item_ID_lists(userIDs,itemIDs):
  i = 0
  item_s = pd.Series()
  user_s = pd.Series()
  for user in userIDs:
    temp = pd.Series(itemIDs)
    item_s = item_s.append(temp)
    temp = []
    temp = [user for item in itemIDs]
    temp = pd.Series(temp)
    user_s = user_s.append(temp)
    progress = user_s.size*100/(userIDs.size * itemIDs.size)
   
    if (progress  > 10) & (int(progress) < 25) & (i == 0):
      print("===== 10 % =====")
      i += 1
    elif (progress > 25) & (int(progress) < 35) & (i == 1):
      print("===== 25 % =====")
      i += 1
    elif (progress > 35) & (int(progress) < 50) & (i == 2):
      print("===== 35 % =====")
      i += 1
    elif (progress > 50) & (int(progress) < 65) & (i == 3):
      print("===== 50 % =====")
      i += 1
    elif (progress > 65) & (int(progress) < 75) & (i == 4):
      print("===== 65 % =====")
      i += 1
    elif (progress > 75) & (int(progress) < 80) & (i == 5):
      print("===== 75 % =====")
      i += 1
    elif (progress > 80) & (int(progress) < 90) & (i == 6):
      print("===== 80 % =====")
      i += 1
    elif (progress > 90) & (int(progress) < 95) & (i == 7):
      print("===== 90 % =====")
      i += 1
    elif (progress > 95) & (i == 8):
      print("===== 95 % =====")
      i += 1

  return user_s,item_s

def Fill_Cf_Matrix(model,userList,itemList,userIDs,itemIDs):

  prediction = model.predict([userList,itemList],verbose = 0)
  i = 0
  row = []
  matrix = []
  print("pred done")

  while i < userList.shape[0]:
    result = np.where(prediction[i] == np.amax(prediction[i]))[0][0] + 1
    row.append(result)
    if len(row) == itemIDs.size:
      matrix.append(row)
      row = []
       
    i += 1

  matrix_arr = np.array(matrix)
  dataframe = pd.DataFrame(matrix_arr, index = userIDs, columns = itemIDs)
  # for user in userIDs:
  #   user_s = pd.Series(user)
  #   for item in itemIDs:
  #     item_s = pd.Series(item)
  #     if dataframe.loc[user][item] == 0:
  #       prediction = model.predict([user_s,item_s],verbose = 0)
  #       result = np.where(prediction[0] == np.amax(prediction[0]))[0][0] + 1
  #       dataframe.loc[user][item] = result
  return dataframe


In [11]:
df_train = preprocess_df(df)

In [ ]:
df_train

In [ ]:
GMF = Create_GMF(df_train['User_ID'].max() + 1,df_train['Item_ID'].max() + 1,64)

In [12]:
df_train_sample = df_train[:75000]
X_train, X_test,y_train,y_test= train_test_split(df_train_sample[['User_ID','Item_ID']],df_train_sample[['one','two','three','four','five','six']],stratify=df_train_sample[['one','two','three','four','five','six']])
#X_train, X_test,y_train,y_test= train_test_split(df_train[['User_ID','Item_ID']],df_train['rating'],stratify=df_train['rating'])

In [ ]:
GMF_trained = Train_GMF(GMF,[X_train['User_ID'],X_train['Item_ID']],y_train,5)

In [13]:
user_item_matrix , df_mat = DatasetToUserItemDataFrame(df_train_sample,'User_ID','Item_ID','rating')

In [ ]:
GMF_trained.evaluate([X_test['User_ID'],X_test['Item_ID']],y_test)

In [ ]:
New_User_IDs = df_train_sample.User_ID.unique()
New_Item_IDs = df_train_sample.Item_ID.unique()
New_User_IDs

In [ ]:
New_Item_IDs.size * New_User_IDs.size

1731900

In [ ]:
user_s,item_s = user_item_ID_lists(New_User_IDs,New_Item_IDs)

In [ ]:
df_mat_filled = Fill_Cf_Matrix(GMF_trained,user_s,item_s,New_User_IDs,New_Item_IDs)

pred done


In [164]:
res = np.where(df_mat.iloc[1,:].values == 5 )
res[0].size


69

In [14]:
df_mat_filled1 = pd.read_csv('/content/epinions_user_item_filled_75K_matrix_100k_leaning.csv')

In [ ]:
df_mat

In [ ]:
df_mat_filled1

In [ ]:
df_mat_sample = df_mat_filled1.iloc[:,:4030]

In [ ]:

df_mat_sample.replace(0,' ', inplace = True)


In [15]:
df_mat_sample =df_mat_filled1

In [16]:
#Autoencoder

encoder_input = layers.Input(shape=(df_mat_sample.shape[1]),name='user_item')
flat = layers.Flatten()(encoder_input)
dropout = layers.Dropout(.2)(flat)
hid_encoder = layers.Dense(512,activation="relu")(dropout)
# hid_encoder1 = layers.Dense(256,activation="relu")(hid_encoder)
# hid_encoder2 = layers.Dense(128,activation="relu")(hid_encoder1)
# hid_encoder3 = layers.Dense(10,activation="relu")(hid_encoder2)
encoder_output = layers.Dense(256,activation="relu")(hid_encoder)

decoder_input = layers.Dense(512,activation="relu")(encoder_output)
# decoder_hidden1 = layers.Dense(256,activation="relu")(decoder_input)
# decoder_hidden2 = layers.Dense(512,activation="relu")(decoder_hidden1)
# decoder_hidden3 = layers.Dense(25,activation="relu")(decoder_hidden2)
decoder_output = layers.Dense(df_mat_sample.shape[1],activation="relu")(decoder_input)

autoencoder = tf.keras.Model(inputs = encoder_input, outputs = decoder_output)

In [17]:
def rmse (y_true,y_pred):
  y_pred = tf.cast(y_pred, tf.float32)
  y_true = tf.cast(y_true, tf.float32)
  rmse = y_true - y_pred
  rmse = tf.square(rmse)
  rmse = tf.math.reduce_mean(rmse)
  rmse = tf.math.sqrt(rmse)
  return rmse

opt = tf.keras.optimizers.Adam(learning_rate=0.00001)

In [18]:
autoencoder.compile(optimizer ='adam',
                    loss = rmse,
                    metrics=['accuracy'])

In [ ]:
autoencoder.summary()

In [19]:
X_train , X_test = train_test_split(df_mat_sample)

In [ ]:
X_test

In [ ]:
autoencoder.fit(X_train, X_train, epochs = 400)

In [ ]:
autoencoder.evaluate(X_test,X_test)

In [ ]:
df_mat

In [34]:
mat_pred = autoencoder.predict(df_mat)

In [ ]:
mat_pred

In [36]:
df_mat_pred = pd.DataFrame(mat_pred, index = df_train_sample.User_ID.unique(), columns = df_train_sample.Item_ID.unique())

In [ ]:
df_mat_pred

In [44]:
df_mat_pred = df_mat_pred.apply(round)

In [ ]:
res = np.where(df_mat_pred.iloc[:,:].values == 1 )
res[0].size